# Dim Calendario

In [0]:
import pyspark.sql.functions as f 

In [0]:
spark.sql('USE CATALOG saude_sus')

In [0]:
start_date = "2026-01-01"
end_date = "2050-12-31"

In [0]:
df_calendario = spark.range(1).select(
    f.explode(
        f.sequence(
            f.to_date(f.lit(start_date)), 
            f.to_date(f.lit(end_date)), 
            f.expr("interval 1 day")
        )
    ).alias("DAT_REFERENCIA")
)

dim_calendario_final = df_calendario.select(
    # Surrogate Key
    f.regexp_replace(f.col("DAT_REFERENCIA"), "-", "").cast("int").alias("SK_CALENDARIO"),
    
    f.col("DAT_REFERENCIA").alias("DAT_REFERENCIA"),
    
    f.year("DAT_REFERENCIA").alias("DAT_ANO"),
    f.month("DAT_REFERENCIA").alias("DAT_MES"),
    f.dayofmonth("DAT_REFERENCIA").alias("DAT_DIA"),
    f.quarter("DAT_REFERENCIA").alias("DAT_TRIMESTRE"),
    f.weekofyear("DAT_REFERENCIA").alias("DAT_SEMANA_ANO"),
    f.dayofweek("DAT_REFERENCIA").alias("DAT_DIA_SEMANA_NUM"),  # 1=Domingo, 7=Sábado
    f.dayofyear("DAT_REFERENCIA").alias("DAT_DIA_ANO"),
    
    f.date_format("DAT_REFERENCIA", "MMMM").alias("NOM_MES"),
    f.date_format("DAT_REFERENCIA", "EEEE").alias("NOM_DIA_SEMANA"),
    f.date_format("DAT_REFERENCIA", "MMM").alias("NOM_MES_ABREV"),
    f.date_format("DAT_REFERENCIA", "EEE").alias("NOM_DIA_SEMANA_ABREV"),
    
    f.concat(
        f.year("DAT_REFERENCIA"), 
        f.lit("Q"), 
        f.quarter("DAT_REFERENCIA")
    ).alias("NOM_ANO_TRIMESTRE"),
    
    f.date_format("DAT_REFERENCIA", "yyyy-MM").alias("NOM_ANO_MES"),
    
    f.when(f.dayofweek("DAT_REFERENCIA").isin([1, 7]), 1).otherwise(0).alias("FLG_FIM_SEMANA"),
    f.when(f.dayofmonth("DAT_REFERENCIA") == 1, 1).otherwise(0).alias("FLG_PRIMEIRO_DIA_MES"),
    f.when(
        f.dayofmonth("DAT_REFERENCIA") == f.dayofmonth(f.last_day("DAT_REFERENCIA")), 
        1
    ).otherwise(0).alias("FLG_ULTIMO_DIA_MES"),
    f.when(f.month("DAT_REFERENCIA").isin([1, 2, 3]), 1).otherwise(0).alias("FLG_PRIMEIRO_TRIMESTRE"),
    
    f.trunc("DAT_REFERENCIA", "month").alias("DAT_PRIMEIRO_DIA_MES"),
    f.last_day("DAT_REFERENCIA").alias("DAT_ULTIMO_DIA_MES"),
    
    f.when(f.month("DAT_REFERENCIA") <= 6, 1).otherwise(2).alias("DAT_SEMESTRE"),
    f.concat(
        f.year("DAT_REFERENCIA"),
        f.lit("S"),
        f.when(f.month("DAT_REFERENCIA") <= 6, 1).otherwise(2)
    ).alias("NOM_ANO_SEMESTRE")
)

display(dim_calendario_final)
dim_calendario_final.write.format("delta").mode("overwrite").saveAsTable("saude_sus.refined.DIM_CALENDARIO")